#### Experimento basado en el 6.1, usando el otro split que divide el set de prueba y el de entrenamiento para predecir cosas del futuro con cosas del pasado.

In [41]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import Imputer

import matplotlib.pyplot as plt
import utils.encodeador as my_encoder
import utils.featurizerV2 as ft_v2

import re

from math import sqrt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")


In [43]:
df_test = pd.read_csv("data/test.csv")

In [44]:
train = ft_v2.featurizerv2(train,"train")
test = ft_v2.featurizerv2(test,"test")

In [45]:
train.shape

(240000, 55)

In [46]:
test.shape

(60000, 54)

In [47]:
drop_cols = ['id','titulo','descripcion','direccion','fecha']

In [48]:
X = train.drop(['precio'] + drop_cols, axis=1)
test = test.drop(drop_cols, axis=1)
y = train['precio']

In [49]:
X,test = my_encoder.oneHotEncoder(X,test)

In [50]:
X.shape

(240000, 631)

In [51]:
value = 2

In [52]:
tss = TimeSeriesSplit(n_splits = value)

In [53]:
for train_index, test_index in tss.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [54]:
X_train.shape

(160000, 631)

In [55]:
X_test.shape

(80000, 631)

In [56]:
y_train.shape

(160000,)

In [57]:
y_test.shape

(80000,)

In [58]:
hyper_param = { 'objective' :'reg:squarederror', 
                'colsample_bytree' : 0.3, 
               'learning_rate' : 0.1,
                'max_depth' : 15,
               'alpha' : 10,
               'n_estimators' : 300,
               'eval_metric' : 'mae'
              }

In [59]:
xgb = XGBRegressor(**hyper_param)

In [60]:
%%time
xgb.fit(X_train, y_train)

/home/braianvillalba/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


CPU times: user 12min 17s, sys: 1.72 s, total: 12min 19s
Wall time: 12min 19s


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, eval_metric='mae',
             gamma=0, importance_type='gain', learning_rate=0.1,
             max_delta_step=0, max_depth=15, min_child_weight=1, missing=None,
             n_estimators=300, n_jobs=1, nthread=None,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=1, verbosity=1)

In [61]:
pred = xgb.predict(X_test)

In [62]:
mean_absolute_error(y_test,pred)

547955.7430469727

In [63]:
predicion = xgb.predict(test)

In [65]:
resultado = pd.DataFrame(predicion, index=df_test.id, columns=['target'])
display(resultado.head())
resultado.to_csv("data/workshop-xgboost-v6_2.csv", header=True)

,target
id,
4941,6.089232e+06
51775,9.939081e+05
115253,2.363573e+06
299321,1.464984e+06
173570,5.997882e+05
